In [1]:
import numpy as np

In [2]:
B_S = 10
Emb_dim = 300
Sequence_length = 50
vocab_size = 30000

## Define Inputs in shape of (Batch_size * Sequence_Length, Dim) and targets as (Batch_size * Sequence_length,1)

In [124]:
X_input = np.random.randn(B_S*Sequence_length,Emb_dim)
target = np.random.randint(30000,size=(B_S*Sequence_length))

## Define the intervals on which we are going to divide our vocabulary accordingly , This mean that the first cluster contains [15000 - 3000] vocab words and the second cluster contains [vocab_size - 15000]

In [115]:
cutoffs = [3000,15000,30000]

## Now we divided our vocabulary into 3 parts , 0 - > 3000 , 3000 - > 15000 , 15000 - > 30000 so we need to define for each part a standalone matrix as each one has a different range of outputs. In addition , the paper proposed to use smaller dimensions for infrequent words as they wont be learned that much.

In [116]:
head_vocab_size = cutoffs[0]
cluster1_vocab_size = cutoffs[1] - cutoffs[0]
cluster2_vocab_size = vocab_size - cutoffs[1]
cluster_number = 2

In [117]:
cluster1_dim = int((300) / (4 ** 1))
cluster2_dim = int((300) / (4 ** 2))

In [ ]:
head_matrix = np.random.randn(Emb_dim,head_vocab_size + cluster_number)
cluster1_matrix1 = np.random.randn(Emb_dim,cluster1_dim)
cluster1_matrix2 = np.random.randn(cluster1_dim,cluster1_vocab_size)
cluster2_matrix1 = np.random.randn(Emb_dim,cluster2_dim)
cluster2_matrix2 = np.random.randn(cluster2_dim,cluster2_vocab_size)

In [126]:
cluster1_matrix1 = np.random.randn(Emb_dim,cluster1_dim)
cluster1_matrix2 = np.random.randn(cluster1_dim,cluster1_vocab_size)
cluster2_matrix1 = np.random.randn(Emb_dim,cluster2_dim)
cluster2_matrix2 = np.random.randn(cluster2_dim,cluster2_vocab_size)
def cluster(i,Input):
    if i == 1:
            Input = np.dot(Input,mat1)
            Input = np.dot(Input,mat2)
            return Input
    elif i == 2 :
            Input = np.dot(Input,mat1)
            Input = np.dot(Input,mat2)
            return Input

In [127]:
new_cutoffs = [0] + cutoffs

In [128]:
## So we are now ready to compute the losses. We have the target matrix which contains indices
## belong to different clusters so lets see how to deal with that

In [135]:
print(target)

[26874 21391 16466   507 22674 18674 22287 24051 29095 10367 23691 14886
 12244  6110 20319 28853  9260  8746 14306 18284 23426 29480 14299 26733
 16507 19382 22305 10199 23307 18690  4560 12551 29480 21871 20814 10352
  3720  4913 27695  9370 25389 24032 10075 12710 17998 20905 14119 26364
 17139 28562  7849 24846  6464 29565  5573 29313 23856 12157  8681 29755
  7587 19752 24238 13645  1176 14143  7560 19402 11730 23278 25255   978
 26121  9755 22263 27574  6633  8067 28166 12929  1513 29407  3356   813
  2964 27471  1689 19708 11114  4370 20050  7207 10128  9576 21349 11706
 10526  9712 11260 18403 14340  6352 29654  2972  3252 13193 23005  4334
 20181 28342 23952  8974  5054 22236  5349 10045  6033  3814 10059   542
 11604 11029  5426 26082 16168  4334 12373 11154  9255 17833 16588 18431
  9726  7552 13700 20960  3662 27920 25976  5727 14151 21089 26341 17256
 23031 21076 12629 20244 22729 18267 16959 11264 23309 22453 15029  1279
  1614 17871 19878 27342  5356  2962 16880  5129  2

In [136]:
def get_loss(X_Input,target):
    # Create an empty matrix to keep track of indices and losses for each entry
    Indices_track = np.empty((B_S*Sequence_length))
    Output_Loss = np.empty((B_S*Sequence_length))
    
    # We are dealing with different numbers of clusters each indexed by certain limits , so we dont care about other indices
    # each iteration
    for i in range(len(new_cutoffs)-1):
        low_boundary = new_cutoffs[i]
        high_boundary = new_cutoffs[i+1]
        Row_mask = (target >= low_boundary) & (target < high_boundary)
        Current_Iteration_indices = np.argwhere(Row_mask==True)
        Current_Iteration_indices = np.squeeze(Current_Iteration_indices.T,axis=0)
        Current_Iteration_values = target[Row_mask]
        
        if i == 0:
            # Keeping track of elements belonging to the head to compute later
            Indices_track[Row_mask] = Current_Iteration_values
        else:
            # Now we are in clusters calculation , which doesnot start index from 0 , so we need to create relative indices
             #   print(Current_Iteration_values)
            relative_indices = Current_Iteration_values - low_boundary
            cluster_results = cluster(i,X_Input[Current_Iteration_indices,:])
            cluster_results = log_softmax(cluster_results)
            for j,(val,indx) in enumerate(zip(relative_indices,Current_Iteration_indices)):
                Output_Loss[indx] = cluster_results[j][val]
                Indices_track[indx] = head_vocab_size + i - 1  
    Head_output = np.dot(X_Input,head_matrix)
    relative_results = np.empty((B_S*Sequence_length))
    Head_output = log_softmax(Head_output)
    for l,k in enumerate(Indices_track):
        relative_results[l] = Head_output[l][int(k)]
    Output_Loss += relative_results
    Mean_Loss = np.mean(-Output_Loss)
    return Output_Loss,Mean_Loss

In [142]:
def log_softmax(x):
    e_x = np.exp(x)
    return np.log(e_x/e_x.sum(axis=1,keepdims=True))